In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

if not es.ping():
    raise Exception("Erreur de connexion à Elasticsearch!")

response = es.search(
    index="adresses",
    body={
        "query": {
            "match": {
                "ville": "Besançon"
            }
        }
    }
)



print("Documents trouvés pour la ville 'Abbans' :")
for doc in response['hits']['hits']:
    print(doc['_source'])


Documents trouvés pour la ville 'Abbans' :
{'id': '250560012-10A', 'numero': '10A', 'rue': 'Rue Abraham Louis Breguet', 'code_postal': '25000', 'ville': 'Besançon', 'source': 'OSM', 'latitude': 47.23479, 'longitude': 5.973675}
{'id': '250560012-10B', 'numero': '10B', 'rue': 'Rue Abraham Louis Breguet', 'code_postal': '25000', 'ville': 'Besançon', 'source': 'OSM', 'latitude': 47.234761, 'longitude': 5.973625}
{'id': '250560012-10C', 'numero': '10C', 'rue': 'Rue Abraham Louis Breguet', 'code_postal': '25000', 'ville': 'Besançon', 'source': 'OSM', 'latitude': 47.234788, 'longitude': 5.973569}
{'id': '250560012-10D', 'numero': '10D', 'rue': 'Rue Abraham Louis Breguet', 'code_postal': '25000', 'ville': 'Besançon', 'source': 'OSM', 'latitude': 47.234779, 'longitude': 5.973536}
{'id': '250560012-10E', 'numero': '10E', 'rue': 'Rue Abraham Louis Breguet', 'code_postal': '25000', 'ville': 'Besançon', 'source': 'OSM', 'latitude': 47.234736, 'longitude': 5.973492}
{'id': '250560012-10F', 'numero':

In [8]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Récupérer toutes les données de l'index 'adresses'
response = es.search(index="adresses", body={"query": {"match_all": {}}, "size": 10000})  # Limite à 10 000 résultats
data = response['hits']['hits']

# Supprimer l'index reformated s'il existe déjà
es.indices.delete(index='adresses_reformated', ignore=[400, 404])

# Définir le nouveau mapping avec le champ geo_point
mapping = {
    "mappings": {
        "properties": {
            "code_postal": {"type": "keyword"},
            "id": {"type": "keyword"},
            "latitude": {"type": "float"},
            "longitude": {"type": "float"},
            "numero": {"type": "text"},
            "rue": {"type": "text"},
            "source": {"type": "text"},
            "ville": {"type": "text"},
            "location": {"type": "geo_point"}  # Champ geo_point
        }
    }
}

# Créer le nouvel index
es.indices.create(index="adresses_reformated", body=mapping)

# Reformater et réindexer les données
for doc in data:
    source = doc['_source']
    # Ajouter le champ geo_point
    if 'latitude' in source and 'longitude' in source:
        source['location'] = {"lat": source['latitude'], "lon": source['longitude']}
    
    # Réindexer dans le nouvel index
    es.index(index="adresses_reformated", body=source, id=doc['_id'])

print("Réindexation terminée.")


C:\Users\mehdi.mougin\AppData\Local\Temp\ipykernel_12200\1666774118.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='adresses_reformated', ignore=[400, 404])


Réindexation terminée.
